# Notebook for unsupervised ML for conservation laws

In [ ]:
# Code taken and adapted from: https://github.com/KindXiaoming/aipoincare/blob/master/backend/training.py
# Dataset from: https://github.com/KindXiaoming/aipoincare/blob/master/backend/data/oned_harmonic
# ChatGPT assistance: https://chatgpt.com/share/6882476e-0d50-8011-b66e-4d9e376e7696

# In case some packages in correct version should be installed, there is a requirements.txt in working_tree, and a code cell below for "!pip install ..."

In [2]:
# ------------------------------------------------------------
# 1a. Mount Google Drive (do this once per Colab session)
# ------------------------------------------------------------
import pickle
from google.colab import drive
drive.mount('/content/drive/')
dir_path = '/content/drive/MyDrive/AI-in-physics_AJP_2025/Conservation-laws/' # needs to be adapted to specific context

Mounted at /content/drive/


In [ ]:
# ------------------------------------------------------------
# 1b. If you are on a local machine, set dir to current working tree
# ------------------------------------------------------------
import pickle
dir_path = '../Conservation-laws/' # needs to be adapted to specific context

In [ ]:
# ------------------------------------------------------------
# 1c. If you use the binder version
# ------------------------------------------------------------

dir_path = '' # needs to be adapted to specific context

In [3]:
# --------------------- Import necessary libraries ---------------------
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import os
from sklearn.decomposition import PCA
import torch.optim as optim
import copy

seed = 0
torch.manual_seed(seed)
np.random.seed(seed)

In [1]:
# --------------------- Set hyperparameters and data config ---------------------

# Whether to remove PCA components below a noise threshold or keep them (normalize instead)
prepcaremove = 'no'  # 'no' = normalize all components, 'yes' = discard low-variance ones
# Variance threshold for PCA dimension filtering
noise_threshold = 0.001
# Hidden layer widths for the neural network; first and last layer are added later
nn_widths = [256, 256]  # Two hidden layers: 64 units and 32 units
hidden_depth = len(nn_widths)
# Negative slope for LeakyReLU activation function
slope = 0.1
# List of noise magnitudes (sigmal) to test during training
sigmals = [1e-3, 1e-2, 3e-2, 1e-1, 3e-1, 1.0, 10.0]
# Optimizer type: 'Adam' or 'SGD'
opt = 'Adam'
# Learning rate for optimizer
lr = 0.001
# Mini-batch size for training
batch_size = 64
# Number of training iterations (epochs)
epoch = 5000
# ERD parameter `a` for computing effective dimensionality
a = 2
# Path to the input data file (must be a .txt file of shape [n_samples, n_features])
model = 'harmonic_1d.txt'
# Number of random walks to simulate for post-training variance analysis
n_walk = 600
# Frequency of logging during training
log = 200

In [4]:
# --------------------- Load and preprocess the data ---------------------

# Load dataset as NumPy array
xs = np.loadtxt(os.path.join(dir_path, model).replace("\\", "/"))
n_train = xs.shape[0]       # Number of samples
input_dim = xs.shape[1]     # Dimensionality of input features

# Step 1: Normalize each input feature to zero mean and unit variance
xs = (xs - np.mean(xs, axis=0)[np.newaxis, :]) / np.std(xs, axis=0)[np.newaxis, :]

# Step 2: Apply PCA (rotation only, no scaling) to decorrelate dimensions
pca = PCA()
xs = pca.fit_transform(xs)

# Step 3: Remove or normalize dimensions based on explained variance
remove_dim = 0
if prepcaremove == "no":
    # Keep all dimensions, normalize each one by variance + threshold
    xs = xs / (np.std(xs, axis=0)[np.newaxis, :] + noise_threshold)
else:
    # Keep only dimensions where variance > threshold
    input_dim_orig = input_dim
    input_dim = np.sum(pca.explained_variance_ratio_ > noise_threshold)
    remove_dim = input_dim_orig - input_dim
    xs = xs[:, :input_dim]
    xs = xs / (np.std(xs, axis=0)[np.newaxis, :])

# Add input/output layers to the neural network layer list
nn_widths.insert(0, input_dim)
nn_widths.append(input_dim)

In [5]:
# --------------------- Training loop across different noise levels ---------------------
exps = []   # To store explained variance ratios
losses = [] # To record loss values during training

for sigmal in sigmals:
    # Define a new neural network (as a list of layers) for each noise level
    linears = [nn.Linear(nn_widths[i], nn_widths[i + 1]) for i in range(hidden_depth + 1)]
    parameters = [p for layer in linears for p in layer.parameters()]  # Collect all trainable parameters

    # Define loss function
    criterion = nn.MSELoss()

    # Choose optimizer
    optimizer = optim.Adam(parameters, lr=lr) if opt == "Adam" else optim.SGD(parameters, lr=lr)

    print(f"sigmal = {sigmal}")

    # --------------------- Training the denoising network ---------------------
    for j in range(epoch):
        # Randomly sample a batch of training data
        choices = np.random.choice(n_train, batch_size)
        perturb = torch.normal(0, sigmal, size=(batch_size, input_dim))  # Add Gaussian noise
        inputs0 = torch.tensor(xs[choices], dtype=torch.float) + perturb  # Noisy input

        # Forward pass through each layer using LeakyReLU
        x = inputs0
        act = nn.LeakyReLU(slope)
        for i in range(hidden_depth):
            x = act(linears[i](x))
        outputs = linears[hidden_depth](x)  # Final layer (no activation)

        # Training objective: recover the *negative* of the noise added
        loss = criterion(outputs, -perturb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(float(loss.data))

        if j % log == 0:
            print(f"Epoch {j} | Loss: {loss:.4f}")

    # --------------------- Perform stochastic walks using trained network ---------------------
    x0 = copy.deepcopy(xs[int(n_train / 2)])  # Start from a central data point
    x0 = x0[np.newaxis, :]
    x0 = x0 + np.random.randn(n_walk, input_dim) * sigmal  # Add noise

    # Pass walk samples through trained denoising network
    x = torch.tensor(x0, dtype=torch.float)
    for i in range(hidden_depth):
        x = act(linears[i](x))
    x = linears[hidden_depth](x)
    x0 = x0 + x.detach().numpy()  # Apply learned correction

    # --------------------- Compute explained variance ratio via PCA ---------------------
    pca = PCA()
    pca.fit(x0)
    svs = pca.singular_values_
    exp_ratio = svs**2 / np.sum(svs**2)
    exps.append(exp_ratio)

# Stack results into a NumPy array
exps = np.array(exps)

sigmal = 0.001
Epoch 0 | Loss: 0.0180
Epoch 200 | Loss: 0.0000
Epoch 400 | Loss: 0.0000
Epoch 600 | Loss: 0.0000
Epoch 800 | Loss: 0.0000
Epoch 1000 | Loss: 0.0000
Epoch 1200 | Loss: 0.0000
Epoch 1400 | Loss: 0.0000
Epoch 1600 | Loss: 0.0000
Epoch 1800 | Loss: 0.0000
Epoch 2000 | Loss: 0.0000
Epoch 2200 | Loss: 0.0000
Epoch 2400 | Loss: 0.0000
Epoch 2600 | Loss: 0.0000
Epoch 2800 | Loss: 0.0000
Epoch 3000 | Loss: 0.0000
Epoch 3200 | Loss: 0.0000
Epoch 3400 | Loss: 0.0000
Epoch 3600 | Loss: 0.0000
Epoch 3800 | Loss: 0.0000
Epoch 4000 | Loss: 0.0000
Epoch 4200 | Loss: 0.0000
Epoch 4400 | Loss: 0.0000
Epoch 4600 | Loss: 0.0000
Epoch 4800 | Loss: 0.0000
sigmal = 0.01
Epoch 0 | Loss: 0.0053
Epoch 200 | Loss: 0.0001
Epoch 400 | Loss: 0.0002
Epoch 600 | Loss: 0.0002
Epoch 800 | Loss: 0.0001
Epoch 1000 | Loss: 0.0002
Epoch 1200 | Loss: 0.0001
Epoch 1400 | Loss: 0.0001
Epoch 1600 | Loss: 0.0001
Epoch 1800 | Loss: 0.0001
Epoch 2000 | Loss: 0.0001
Epoch 2200 | Loss: 0.0001
Epoch 2400 | Loss: 0.00

In [6]:
# --------------------- Define helper function to compute effective rank (ERD) ---------------------
def f(x, a=2):
    n = x.shape[1]
    mask = x < 1 / (a * n)
    return np.sum(np.cos(np.pi / 2 * n * a * x) * mask, axis=1)

# --------------------- Plot explained variance and effective dimensionality ---------------------
plt.figure(figsize=(7, 5))

# Plot each PCA component’s explained variance
for i in range(input_dim):
    plt.plot(sigmals, exps[:, i], marker="o")

plt.xscale('log')
plt.xlabel(r"$L$", fontsize=25)
plt.ylabel("Explained Ratio", fontsize=25)

# Plot effective dimensionality (ERD) on second y-axis
ax2 = plt.gca().twinx()
ax2.plot(sigmals, f(exps, a=2), marker="o", color="red", linewidth=5, ls="--", markersize=15)
plt.ylabel(r"$m_{eff}$", fontsize=25, color="red")

# Save the plot to disk
plt.savefig(os.path.join(dir_path, 'ERD.png').replace("\\", "/"), bbox_inches="tight")
plt.clf()

/tmp/ipython-input-6-4039673660.py:25: UserWarning: Attempt to set non-positive xlim on a log-scaled axis will be ignored.
  plt.clf()


<Figure size 700x500 with 0 Axes>